In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from pathlib import Path
import geopandas as gpd
from shapely.geometry import Point

# ===================== CONFIGURAÇÕES =====================
ARQUIVO_ENTRADA = Path(r"C:\Users\vitor.braz\Downloads\geolocalizacao_cet_flask\dados\2024-2025_fatais_motos_05_meses.xlsx")
ARQUIVO_SAIDA = ARQUIVO_ENTRADA.with_name("2024-2025_fatais_motos_05_meses_COM_COORDENADAS_SHAPE_V3.xlsx")
SHAPEFILE_PATH = r"C:\Users\vitor.braz\Downloads\geolocalizacao_cet_flask\dados\vwBlCET_det_get_distrito_sub_4326\vwBlCET_DET_GET_DIST_SUB_4326.shp"

# ===================== TIPOS DE VIA PARA REMOÇÃO =====================
TIPOS_VIA = [
    # Versões por extenso
    "ACESSO", "ALAMEDA", "AVENIDA", "BECO", "CAMINHO", "COMPLEXO VIARIO", "ESPACO LIVRE", "ESPLANADA",
    "ESTRADA", "ESCADARIA", "ESTRADA PARTICULAR", "GALERIA", "LADEIRA", "LARGO", "PASSARELA", "PRACA",
    "PRACA PROJETADA", "PARQUE", "PARQUE ESTADUAL", "PARQUE LINEAR", "PARQUE MUNICIPAL",
    "PASSAGEM DE PEDESTRE", "PASSAGEM PARTICULAR", "PASSAGEM SUBTERRÂNEA", "PONTE", "PONTILHAO",
    "RUA", "RUA PARTICULAR", "RUA PROJETADA", "RODOVIA", "TUNEL", "TRAVESSA", "TRAVESSA PARTICULAR",
    "VIA DE CIRCULACAO DE PEDESTESTRES", "VIADUTO", "VIELA", "VIA ELEVADA", "VIA ELEVADA DE PEDESTRES",
    "VEREDA", "VIELA SANITARIA", "VILA", "VIELA PROJETADA", "VIELA PARTICULAR","ESTRADA DO ",
    "COMPLEXO VIARIO", "RUA DA ","PRAÇA","TÚNEL"   
    # Abreviações comuns
    "AC", "AL", "AV", "BC", "CM", "CV", "EL", "EPL", "ES", "ESC", "ESP", "GL", "LD", "LG",
    "PA", "PC", "PP", "PQ", "PQE", "PQL", "PQM", "PS", "PSP", "PSS", "PT", "PTE", "PTL",
    "R", "RP", "RPJ", "RV", "TN", "TV", "TVP", "VCP", "VD", "VE", "VEL", "VEP", "VER", "VES",
    "VL", "VLP", "VP", "ES. DO", "ES DO"
]

def remover_tipo_via(endereco: str) -> str:
    partes = endereco.strip().upper().split()
    if partes and partes[0] in TIPOS_VIA:
        return " ".join(partes[1:]).strip()
    return endereco.strip()

# ===================== FUNÇÃO RAW DE BUSCA =====================
def _buscar_enderecos_raw(endereco: str) -> pd.DataFrame:
    url = "http://cetaplica/geosoap/geocode.asmx/buscaEnderecos"
    try:
        response = requests.get(url, params={"endereco": endereco}, timeout=5)
        response.raise_for_status()
    except requests.RequestException:
        return pd.DataFrame()

    try:
        ns = {"ns": "http://tempuri.org/"}
        root = ET.fromstring(response.content)
        enderecos = root.findall("ns:Endereco", ns)
        registros = []
        for endereco_xml in enderecos:
            log = endereco_xml.find("ns:logradouro1", ns)
            if log is not None:
                registros.append({
                    "codlog": log.findtext("ns:codlog", default="", namespaces=ns),
                    "tipo": log.findtext("ns:tipo", default="", namespaces=ns),
                    "titulo": log.findtext("ns:titulo", default="", namespaces=ns),
                    "preposicao": log.findtext("ns:preposicao", default="", namespaces=ns),
                    "nome": log.findtext("ns:nome", default="", namespaces=ns),
                    "distrito": log.findtext("ns:distrito", default="", namespaces=ns),
                })
        return pd.DataFrame(registros)
    except ET.ParseError:
        return pd.DataFrame()

# ===================== BUSCA COM FALLBACK =====================
def buscar_enderecos(endereco: str) -> pd.DataFrame:
    df = _buscar_enderecos_raw(endereco)
    if df.empty:
        # Tenta sem tipo de via
        endereco_sem_tipo = remover_tipo_via(endereco)
        if endereco_sem_tipo and endereco_sem_tipo != endereco.upper():
            print(f"[INFO] Tentando sem tipo de via: '{endereco_sem_tipo}'")
            df = _buscar_enderecos_raw(endereco_sem_tipo)
    return df

# ===================== RESTANTE DAS FUNÇÕES =====================

def logradouro_pmsp(row) -> str:
    partes = [
        (row.get("tipo") or "").strip(),
        (row.get("titulo") or "").strip(),
        (row.get("preposicao") or "").strip(),
        (row.get("nome") or "").strip()
    ]
    return " ".join(p for p in partes if p).strip()

def montar_endereco_completo(row, numero="") -> str:
    return f"{row['codlog']}, {numero}".strip()

# ===================== BUSCA LAT/LON EXATO =====================
def buscar_latlon_exato(endereco: str) -> tuple[float, float] | None:
    url = "http://cetaplica/geosoap/geocode.asmx"
    headers = {
        "Content-Type": "text/xml; charset=utf-8",
        "SOAPAction": "http://tempuri.org/buscaLatLonExato"
    }
    soap_body = f"""<?xml version="1.0" encoding="utf-8"?>
    <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
                   xmlns:xsd="http://www.w3.org/2001/XMLSchema"
                   xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
      <soap:Body>
        <buscaLatLonExato xmlns="http://tempuri.org/">
          <endereco>{endereco}</endereco>
        </buscaLatLonExato>
      </soap:Body>
    </soap:Envelope>"""

    try:
        response = requests.post(url, data=soap_body, headers=headers, timeout=5)
        response.raise_for_status()
        ns = {"soap": "http://schemas.xmlsoap.org/soap/envelope/", "ns": "http://tempuri.org/"}
        root = ET.fromstring(response.content)
        ponto = root.find(".//ns:buscaLatLonExatoResult", ns)
        if ponto is None:
            return None
        x = ponto.findtext("ns:x", default=None, namespaces=ns)
        y = ponto.findtext("ns:y", default=None, namespaces=ns)
        if x is None or y is None:
            return None
        return float(y), float(x)
    except:
        return None

# ===================== SHAPEFILE =========================
gdf_shp = gpd.read_file(SHAPEFILE_PATH).to_crs("EPSG:31983")
CAMPOS_SHAPE = {
    'logradouro': 'logradouro_shp',
    'geometry': 'geometry_shp',
    'lg_codlog': 'lg_codlog_shp',
    'tipo_via': 'tipo_via_shp',
    'local': 'local_shp',
    'det_sigla': 'det_sigla_shp',
    'get': 'get_shp',
    'distrito': 'distrito_shp',
    'sub': 'sub_shp',
    'regiao': 'regiao_shp'
}

def buscar_no_shapefile(lat, lon):
    try:
        ponto = gpd.GeoSeries([Point(float(lon), float(lat))], crs="EPSG:4326").to_crs("EPSG:31983").iloc[0]
        gdf_shp["dist"] = gdf_shp.distance(ponto)
        candidato = gdf_shp.sort_values("dist").iloc[0]
        if candidato["dist"] > 50:
            raise ValueError
        dados = {v: candidato[k] for k, v in CAMPOS_SHAPE.items()}
        dados['lat_shp'] = ponto.y
        dados['long_shp'] = ponto.x
        return dados
    except:
        return {v: "NÃO ENCONTRADO" for v in CAMPOS_SHAPE.values()} | {
            'lat_shp': "NÃO ENCONTRADO",
            'long_shp': "NÃO ENCONTRADO"
        }

# ===================== PROCESSAMENTO =====================
df_input = pd.read_excel(ARQUIVO_ENTRADA)
dados_final = []

for _, row in df_input.iterrows():
    logradouro = str(row.get("logradouro", "")).strip()
    numero = str(row.get("numero_logradouro", "")).strip()
    lat_ref = row.get("latitude")
    lon_ref = row.get("longitude")
    resultado = row.to_dict()

    if not logradouro or pd.isna(lat_ref) or pd.isna(lon_ref):
        resultado |= {k: "NÃO ENCONTRADO" for k in [
            "codlog_geo", "logradouro_PMSP_geo", "consulta_geo",
            "latitude_geo", "longitude_geo", "dist_geo_occ"
        ]}
    else:
        df_ends = buscar_enderecos(logradouro)
        if df_ends.empty:
            resultado |= {k: "NÃO ENCONTRADO" for k in [
                "codlog_geo", "logradouro_PMSP_geo", "consulta_geo",
                "latitude_geo", "longitude_geo", "dist_geo_occ"
            ]}
        else:
            ponto_ref = Point(lon_ref, lat_ref)
            min_dist = float('inf')
            melhor = None

            for _, cand in df_ends.iterrows():
                endereco_consulta = montar_endereco_completo(cand, numero)
                latlon = buscar_latlon_exato(endereco_consulta)
                if latlon:
                    p_cand = Point(latlon[1], latlon[0])
                    dist = ponto_ref.distance(p_cand)
                    if dist < min_dist:
                        min_dist = dist
                        melhor = (cand, latlon, endereco_consulta, dist)

            if melhor:
                c, latlon, endereco_consulta, dist = melhor
                resultado.update({
                    "codlog_geo": c["codlog"],
                    "logradouro_PMSP_geo": logradouro_pmsp(c),
                    "consulta_geo": endereco_consulta,
                    "latitude_geo": latlon[0],
                    "longitude_geo": latlon[1],
                    "dist_geo_occ": dist
                })
            else:
                resultado |= {k: "NÃO ENCONTRADO" for k in [
                    "codlog_geo", "logradouro_PMSP_geo", "consulta_geo",
                    "latitude_geo", "longitude_geo", "dist_geo_occ"
                ]}

    if resultado.get("latitude_geo") != "NÃO ENCONTRADO":
        resultado |= buscar_no_shapefile(resultado["latitude_geo"], resultado["longitude_geo"])
    else:
        resultado |= {v: "NÃO ENCONTRADO" for v in CAMPOS_SHAPE.values()} | {
            'lat_shp': "NÃO ENCONTRADO",
            'long_shp': "NÃO ENCONTRADO"
        }

    dados_final.append(resultado)

# ===================== SALVA =============================
df_final = pd.DataFrame(dados_final)
df_final.to_excel(ARQUIVO_SAIDA, index=False)
print(f"\n✅ Arquivo salvo com sucesso: {ARQUIVO_SAIDA}")


C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


[INFO] Tentando sem tipo de via: 'CONDE DE S JOAQUIM'


C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\sha

[INFO] Tentando sem tipo de via: 'DOUTOR ASSIS RIBEIRO'


C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\sha

...

C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\sha

[INFO] Tentando sem tipo de via: 'DAS LAGRIMAS'


C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


[INFO] Tentando sem tipo de via: 'PROFESSOR CARDOZO DE MELLO NETO'


C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
C:\Users\vitor.braz\AppData\Roaming\Python\Python312\site-packages\sha